# EDA / insights on poet vocabulary/complexity/pos/readability

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [2]:
import csv
import numpy as np

## import text files

In [3]:
wilde_quotes = [] # import each text file
with open('texts/wilde_quotes.csv', 'r') as csvfile: 
    reader = csv.reader(csvfile, delimiter='\n')
    for row in reader:
        wilde_quotes.append('\n'.join(row))
        
frost_quotes = [] # import each text file
with open('texts/frost_quotes.txt', 'r') as csvfile: 
    reader = csv.reader(csvfile, delimiter='\n')
    for row in reader:
        frost_quotes.append('\n'.join(row))
        

kaur_quotes = [] # import each text file
with open('texts/kaur_quotes.csv', 'r') as csvfile: 
    reader = csv.reader(csvfile, delimiter='\n')
    for row in reader:
        kaur_quotes.append('\n'.join(row))
        
        
eliot_quotes = [] # import each text file
with open('texts/eliot_quotes.txt', 'r') as csvfile: 
    reader = csv.reader(csvfile, delimiter='\n')
    for row in reader:
        eliot_quotes.append('\n'.join(row))
        
        
shakespeare_quotes = [] # import each text file
with open('texts/shakespeare_quotes.txt', 'r') as csvfile: 
    reader = csv.reader(csvfile, delimiter='\n')
    for row in reader:
        shakespeare_quotes.append('\n'.join(row))

### stopwords

In [4]:
# import stopwords
import nltk
#nltk.download()

from nltk.corpus import stopwords
sw = stopwords.words("english")
print(sw)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## LDA/ Topic Modeling

In [44]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(42)
import nltk
# nltk.download('wordnet')

def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [132]:
# each poet

processed_docs = []

for i in np.arange(len(eliot_quotes)):
    processed_docs.append(preprocess(eliot_quotes[i]))
    
# for i in np.arange(len(frost_quotes)):
#     processed_docs.append(preprocess(frost_quotes[i]))
    
# for i in np.arange(len(kaur_quotes)):
#     processed_docs.append(preprocess(kaur_quotes[i]))
    
# for i in np.arange(len(eliot_quotes)):
#     processed_docs.append(preprocess(eliot_quotes[i]))
    
# for i in np.arange(len(shakespeare_quotes)):
#     processed_docs.append(preprocess(shakespeare_quotes[i]))

processed_docs = list(filter(None, processed_docs)) # remove empty lists  
# print(processed_docs)

In [136]:
dictionary = gensim.corpora.Dictionary(processed_docs)   
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
        
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)    

0 aunt
1 helen
2 maiden
3 miss
4 slingsbi
5 fashion
6 hous
7 live
8 near
9 small
10 squar


In [137]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
#bow_corpus = list(filter(None, bow_corpus)) # remove empty lists, is this useful here?

In [139]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
    
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=4, id2word=dictionary, passes=75, workers=2, minimum_probability=.1)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.078*"time" + 0.067*"hand" + 0.064*"think" + 0.052*"hair" + 0.049*"eye" + 0.035*"wind" + 0.035*"white" + 0.032*"arm" + 0.029*"brown" + 0.024*"see"
Topic: 1 
Words: 0.072*"know" + 0.069*"shall" + 0.047*"turn" + 0.044*"night" + 0.039*"give" + 0.039*"like" + 0.036*"light" + 0.031*"door" + 0.026*"sing" + 0.026*"look"
Topic: 2 
Words: 0.054*"water" + 0.045*"rock" + 0.042*"even" + 0.042*"come" + 0.037*"window" + 0.028*"break" + 0.028*"friend" + 0.026*"weav" + 0.026*"afternoon" + 0.021*"dead"
Topic: 3 
Words: 0.091*"street" + 0.076*"say" + 0.040*"feet" + 0.039*"leav" + 0.034*"room" + 0.031*"face" + 0.031*"fall" + 0.031*"smell" + 0.031*"hous" + 0.031*"lamp"


### Tokenization

In [ ]:
from pprint import pprint
from collections import defaultdict
import string

def get_tokens(doc):
    # remove punctuation
    translator = str.maketrans('', '', string.punctuation) 
    for i in range(len(doc)):
        doc[i] = doc[i].translate(translator).replace('”', '').replace('“', '') # removes smart quotes

    # remove stop words and tokenize
    texts = [[word for word in quote.lower().split() if word not in sw] for quote in doc]

    # remove words that appear only once
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1

    texts = [[token for token in text if frequency[token] > 1]
             for text in texts]

    return(texts)

In [ ]:
wilde_tokens = get_tokens(wilde_quotes)
frost_tokens = get_tokens(frost_quotes)
kaur_tokens = get_tokens(kaur_quotes)
eliot_tokens = get_tokens(eliot_quotes)
shakespeare_tokens = get_tokens(shakespeare_quotes)

In [ ]:
processed_docs = []
for i in np.arange(len(eliot_quotes)):
    str(eliot_quotes[i]).map(preprocess)
processed_docs.append(i)
processed_docs[:10]


processed_docs = documents['headline_text'].map(preprocess)

In [ ]:
from gensim import corpora
import operator

# Create dicts with key:value as “How many times does the word <___> appear in the document? Once.”

wilde_dict = corpora.Dictionary(wilde_tokens)
wilde_dict.save('/tmp/wilde.dict')  # store the dictionary, for future reference
# print(wilde_dict.token2id)
# find most common word
print("most frequent wilde word = " + str(max(wilde_dict.token2id.items(), key=operator.itemgetter(1))))

frost_dict = corpora.Dictionary(frost_tokens)
frost_dict.save('/tmp/frost.dict')
print("most frequent frost word = " + str(max(frost_dict.token2id.items(), key=operator.itemgetter(1))))

kaur_dict = corpora.Dictionary(kaur_tokens)
kaur_dict.save('/tmp/kaur.dict') 
print("most frequent kaur word = " + str(max(kaur_dict.token2id.items(), key=operator.itemgetter(1))))

eliot_dict = corpora.Dictionary(eliot_tokens)
eliot_dict.save('/tmp/eliot.dict') 
print("most frequent eliot word = " + str(max(eliot_dict.token2id.items(), key=operator.itemgetter(1))))

shakespeare_dict = corpora.Dictionary(shakespeare_tokens)
shakespeare_dict.save('/tmp/shakespeare.dict') 
print("most frequent shakespeare word = " + str(max(shakespeare_dict.token2id.items(), key=operator.itemgetter(1))))


### sentiment analysis

In [ ]:
# vader for sentiment, or textblob

In [ ]:
import nltk
import pandas as pd 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [ ]:
def get_sentiment(quotes):
    summary = {"positive":0,"neutral":0,"negative":0}
    for x in quotes: 
        ss = sid.polarity_scores(x)
        if ss["compound"] == 0.0: 
            summary["neutral"] +=1
        elif ss["compound"] > 0.0:
            summary["positive"] +=1
        else:
            summary["negative"] +=1
    return(summary)    

In [ ]:
get_sentiment(wilde_quotes)

In [ ]:
get_sentiment(frost_quotes)

In [ ]:
get_sentiment(kaur_quotes)

In [ ]:
get_sentiment(eliot_quotes)

In [ ]:
get_sentiment(shakespeare_quotes)

### readability scores

In [ ]:
# fleisch-kincaid score, dale-chall

In [ ]:
# https://pypi.org/project/spacy-readability/
import spacy
from spacy_readability import Readability

def get_readability(quotes):
    
    nlp = spacy.load('en')
    read = Readability()
    nlp.add_pipe(read, last=True)

    doc = nlp(quotes)

#     print(doc._.flesch_kincaid_grade_level)
#     print(doc._.flesch_kincaid_reading_ease)
#     print(doc._.dale_chall)
#     print(doc._.smog)
#     print(doc._.coleman_liau_index)
#     print(doc._.automated_readability_index)
#     print(doc._.forcast)

    stats = [doc._.flesch_kincaid_grade_level,doc._.flesch_kincaid_reading_ease,doc._.dale_chall,doc._.smog,doc._.coleman_liau_index,\
             doc._.automated_readability_index, doc._.forcast]
    
    return stats

In [ ]:
def get_scoring_avg(quotes):

    length = len(quotes)
    
    for i in np.arange(length):

        fk_grade_lvl = []
        fk_grade_ease = []
        dc = []
        smg = []
        cli = []
        a_read_indx = []
        forecast = []

        scores = get_readability(quotes[i])
        #print(scores)
        
        fk_grade_lvl.append(scores[0])
        fk_grade_ease.append(scores[1])
        dc.append(scores[2])
        smg.append(scores[3])
        cli.append(scores[4])
        a_read_indx.append(scores[5])
        forecast.append(scores[6])

        avg_fk_grade_lvl = 0
        avg_fk_grade_ease = 0
        avg_dc = 0
        avg_smg = 0
        avg_cli = 0
        avg_a_read_indx = 0
        avg_forecast = 0
        
        max_fk_grade_lvl = 0
        max_fk_grade_ease = 0
        max_dc = 0
        max_smg = 0
        max_cli = 0
        max_a_read_indx = 0
        max_forecast = 0
        
        for n in fk_grade_lvl:
            avg_fk_grade_lvl += n
            avg_fk_grade_lvl = avg_fk_grade_lvl/length
            
            if n > max_fk_grade_lvl:
                max_fk_grade_lvl = n
            
        for n in fk_grade_ease:
            avg_fk_grade_ease += n
            avg_fk_grade_ease = avg_fk_grade_ease/length
            
            if n > max_fk_grade_ease:
                max_fk_grade_ease = n
            
        for n in dc:
            avg_dc += n
            avg_dc = avg_dc/length
            
            if n > max_dc:
                max_dc = n            
            
        for n in smg:
            avg_smg += n
            avg_smg = avg_smg/length
            
            if n > max_smg:
                max_smg = n            
                        
        for n in cli:
            avg_cli += n
            avg_cli = avg_cli/length
            
            if n > max_cli:
                max_cli = n       
            
            
        for n in a_read_indx:
            avg_a_read_indx += n
            avg_a_read_indx = avg_a_read_indx/length
            
            if n > max_a_read_indx:
                max_a_read_indx = n
            
            
        for n in forecast:
            avg_forecast += n
            avg_forecast = avg_forecast/length 
            
            if n > max_forecast:
                max_forecast = n
        
    return ["flesch_kincaid_grade_level = " + str(avg_fk_grade_lvl), "flesch_kincaid_reading_ease = "+ str(avg_fk_grade_ease), \
            "dale_chall = " + str(avg_dc), "smog = " + str(avg_smg), "coleman_liau_index = " + str(avg_cli), \
            "automated_readability_index = " + str(avg_a_read_indx), "forcast = " + str(avg_forecast), \
            "max flesch_kincaid_grade_level = " + str(max_fk_grade_lvl), "max flesch_kincaid_reading_ease = "+ str(max_fk_grade_ease), \
            "max dale_chall = " + str(max_dc), "max smog = " + str(max_smg), "max coleman_liau_index = " + str(max_cli), \
            "max automated_readability_index = " + str(max_a_read_indx), "max forcast = " + str(max_forecast)]

In [ ]:
#get_scoring_avg(wilde_quotes)
#get_scoring_avg(frost_quotes)
#get_scoring_avg(kaur_quotes)
#get_scoring_avg(eliot_quotes)
get_scoring_avg(shakespeare_quotes)


In [ ]:
# # with https://pypi.org/project/textstat/ instead
# import textstat

# test_data = wilde_quotes[1]

# print(textstat.flesch_reading_ease(test_data))
# print(textstat.smog_index(test_data))
# print(textstat.flesch_kincaid_grade(test_data))
# print(textstat.coleman_liau_index(test_data))
# print(textstat.automated_readability_index(test_data))
# print(textstat.dale_chall_readability_score(test_data))
# print(textstat.difficult_words(test_data))
# print(textstat.linsear_write_formula(test_data))
# print(textstat.gunning_fog(test_data))
# print(textstat.text_standard(test_data))


### Tokenization and parts of speech tagging

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
import nltk.tokenize as nt
import nltk

text = wilde_quotes[0]
ss = nt.sent_tokenize(text)
tokenized_sent = [nt.word_tokenize(sent) for sent in ss]
pos_sentences = [nltk.pos_tag(sent) for sent in tokenized_sent]
pos_sentences

In [ ]:
def get_pos(quotes):
    
    
    return []

In [ ]:
# extract parts of speech, store in df, export to csv, add to visualizations

### Syllable count comparison

In [142]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count


def get_syllable_superlatives(quote_list):
    count = 0 # using to avoid calling syllable_count twice
    max_syllables = 0
    total_syllables = 0
    words=0
    ignore = ['Gwendolen–Cecily–it', 'Societycivilized', \
              'Squarethecircletillyouretired', 'somebodydirector', 'blackpapercovered',\
              'roofsPolyphiloprogenitive']
         
    for quote in quote_list:
        for word in quote.split():
            words+=1 # track num words to get avg later
            count = syllable_count(word)
            
            if count > max_syllables:
                if word in ignore: # if a known bogus word, ignore it in our average count
                    words-=1 
                else: # word must be longer and real
                    max_syllables = count
                    
                if word not in ignore:
                    print(word) # sanity check to make sure we don't include concatted words
            total_syllables +=count    
                
    return "max = " + str(max_syllables), "avg = " + str(total_syllables/words)

In [143]:
#get_syllable_superlatives(wilde_quotes)
#get_syllable_superlatives(frost_quotes)
#get_syllable_superlatives(kaur_quotes)
#get_syllable_superlatives(eliot_quotes)
get_syllable_superlatives(shakespeare_quotes)

From
fairest
creatures
self-substantial
determination:


('max = 5', 'avg = 1.2968848242354958')

In [144]:
# syllable_count("superlative")

In [ ]:
#tf-idf 

wilde_corpus = []
for line in wilde_tokens:
    wilde_corpus.append(wilde_dict.doc2bow(line))

frost_corpus = []
for line in frost_tokens:
    frost_corpus.append(frost_dict.doc2bow(line))
      
kaur_corpus = []
for line in kaur_tokens:
    kaur_corpus.append(kaur_dict.doc2bow(line))

eliot_corpus = []
for line in eliot_tokens:
    eliot_corpus.append(eliot_dict.doc2bow(line))    

shakespeare_corpus = []
for line in shakespeare_tokens:
    shakespeare_corpus.append(shakespeare_dict.doc2bow(line))    

from gensim.models import TfidfModel
wilde_tfidf= TfidfModel(wilde_corpus)

frost_tfidf= TfidfModel(frost_corpus)

kaur_tfidf= TfidfModel(kaur_corpus)

eliot_tfidf= TfidfModel(eliot_corpus)

shakespeare_tfidf= TfidfModel(shakespeare_corpus)